### Manipulating inpputs and outputs

- **Using RunnableParallel for Format Manipulation**:
  - `RunnableParallel` is effective for adjusting the output format of one Runnable to align with the input requirements of the next Runnable in a sequence.
  - An example scenario involves a prompt expecting a map input with keys "context" and "question", while the actual user input is only the question.
  - To address this, `RunnableParallel` is used to retrieve the necessary context and simultaneously pass through the user's input under the "question" key.
  - This method ensures that both context and user question are appropriately formatted and ready for subsequent processing in the chain.


- **Ease of Composition with RunnableParallel**:
  - When integrating a `RunnableParallel` with another Runnable in LangChain, it's not necessary to explicitly wrap the dictionary in the `RunnableParallel` class.
  - This is because type conversion is automatically managed within the framework.
  - In a chain context, direct use of a dictionary and its explicit wrapping in `RunnableParallel` are functionally equivalent.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

### Following three examples are the same

In [3]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [4]:
from langchain_core.runnables import RunnableParallel

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [5]:
retrieval_chain = (
    RunnableParallel(context=retriever, question=RunnablePassthrough())
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

### Using `itemgetter` as shorthand

Note that you can use Python’s itemgetter as shorthand to extract data from the map when combining with RunnableParallel.

In [7]:
from operator import itemgetter

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

In [8]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
chain.invoke(
  {
    "question": "where did harrison work",
    "language": "finnish"
  }
)

'Harrison työskenteli Kenshossa.'

### Pararellize steps

 **Functionality of RunnableParallel (RunnableMap)**:
  - `RunnableParallel`, also known as `RunnableMap`, simplifies the process of executing multiple Runnables concurrently in LangChain.
  - It compiles and returns the outputs of these parallel Runnables in the form of a map.


In [10]:
from langchain_core.runnables import RunnableParallel

In [11]:
model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"),
 'poem': AIMessage(content="In forest's embrace, a gentle giant roams,\nMajestic bear, nature's heartbeats he owns.")}